<a href="https://colab.research.google.com/github/mrcheong200/code/blob/main/bubble_insertion_sort.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Colab 進行matplotlib繪圖時顯示繁體中文
# 下載台北思源黑體並命名taipei_sans_tc_beta.ttf，移至指定路徑
!wget -O TaipeiSansTCBeta-Regular.ttf https://drive.google.com/uc?id=1eGAsTN1HBpJAkeVM57_C7ccp7hbgSz3_&export=download

import matplotlib

# 改style要在改font之前
# plt.style.use('seaborn')

matplotlib.font_manager.fontManager.addfont('TaipeiSansTCBeta-Regular.ttf')
matplotlib.rc('font', family='Taipei Sans TC Beta')

--2025-11-15 14:24:44--  https://drive.google.com/uc?id=1eGAsTN1HBpJAkeVM57_C7ccp7hbgSz3_
Resolving drive.google.com (drive.google.com)... 74.125.26.139, 74.125.26.102, 74.125.26.101, ...
Connecting to drive.google.com (drive.google.com)|74.125.26.139|:443... connected.
HTTP request sent, awaiting response... 303 See Other
Location: https://drive.usercontent.google.com/download?id=1eGAsTN1HBpJAkeVM57_C7ccp7hbgSz3_ [following]
--2025-11-15 14:24:44--  https://drive.usercontent.google.com/download?id=1eGAsTN1HBpJAkeVM57_C7ccp7hbgSz3_
Resolving drive.usercontent.google.com (drive.usercontent.google.com)... 173.194.217.132, 2607:f8b0:400c:c08::84
Connecting to drive.usercontent.google.com (drive.usercontent.google.com)|173.194.217.132|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 20659344 (20M) [application/octet-stream]
Saving to: ‘TaipeiSansTCBeta-Regular.ttf’

TaipeiSansTCBeta-Re 100%[===================>]  19.70M   121MB/s    in 0.2s    

2025-11-15 14:24:46

## 氣泡排序法

In [ ]:
import os
from typing import List
import matplotlib.pyplot as plt

def save_bubble_sort_frames(a: List[int], outdir: str = "frames_bubble", ascending: bool = True):
    """
    逐步靜態可視化：氣泡排序（Bubble Sort）
    - 先輸出初始圖
    - 逐步輸出每次比較（並標示是否交換）
    - 每趟結束輸出一張圖，更新右側已就位區域
    - 最後輸出一張「完成」總結圖（全部綠色），顯示總比較次數
    """
    os.makedirs(outdir, exist_ok=True)
    arr = a[:]
    n = len(arr)
    frame_id = 0
    comparisons = 0

    def draw(current, i, j, pass_idx, swapped, settled, title_suffix=""):
        """
        current: 當前陣列快照
        i, j: 當前比較的索引（若為趟結束或初始/完成則為 -1）
        pass_idx: 第幾趟（0-based）
        swapped: 本步是否發生交換（或本趟是否有交換，於 i=j=-1 時解讀）
        settled: 已就位的元素數量（右側綠色）
        title_suffix: 額外標題（｜初始、｜完成）
        """
        nonlocal frame_id
        plt.figure(figsize=(7.5, 4))
        bars = plt.bar(range(len(current)), current, color="#4C78A8")

        # 右側已就位區域標綠色
        for idx in range(len(current) - settled, len(current)):
            if 0 <= idx < len(current):
                bars[idx].set_color("#72B07E")

        # 當前比較的兩根柱子標橙色
        if 0 <= i < len(current):
            bars[i].set_color("#F28E2B")
        if 0 <= j < len(current):
            bars[j].set_color("#F28E2B")

        plt.title("氣泡排序（Bubble Sort）逐步示意" + title_suffix)
        plt.xlabel("索引")
        plt.ylabel("數值")
        ymax = max(current) if current else 1
        plt.ylim(0, ymax * 1.2 if ymax > 0 else 1)

        # 文本資訊
        if title_suffix == "｜初始":
            info = f"初始陣列 | 長度：{len(current)}"
        elif title_suffix == "｜完成":
            info = f"排序完成 | 總比較：{comparisons} | 右側已就位：{settled} 個"
        else:
            if i != -1:
                step = f"第 {pass_idx + 1} 趟 | 比較 {i} 與 {j} | "
                action = "本步有交換" if swapped else "本步無交換"
            else:
                step = f"第 {pass_idx + 1} 趟結束 | "
                action = "本趟有交換" if swapped else "本趟無交換（可提前結束）"
            info = f"{step}累積比較：{comparisons} | {action} | 右側已就位：{settled} 個"
        plt.text(0.02, 0.95, info, transform=plt.gca().transAxes, va='top', fontsize=10)

        plt.tight_layout()
        path = os.path.join(outdir, f"frame_{frame_id:04d}.png")
        plt.savefig(path, dpi=140)
        plt.close()
        frame_id += 1

    # 初始圖
    draw(arr[:], i=-1, j=-1, pass_idx=0, swapped=False, settled=0, title_suffix="｜初始")

    if n <= 1:
        # 0 或 1 個元素，直接完成
        draw(arr[:], i=-1, j=-1, pass_idx=0, swapped=False, settled=n, title_suffix="｜完成")
        return

    # 排序過程
    any_swap_overall = False
    for pass_idx in range(n - 1):
        any_swap = False
        # 在本趟內，上一趟已就位數量是 pass_idx
        for i in range(n - 1 - pass_idx):
            j = i + 1
            comparisons += 1
            need_swap = (arr[i] > arr[j]) if ascending else (arr[i] < arr[j])
            if need_swap:
                arr[i], arr[j] = arr[j], arr[i]
                any_swap = True
                any_swap_overall = True
            # 本步畫面（settled=pass_idx 表示上一趟完成的已就位數）
            draw(arr[:], i=i, j=j, pass_idx=pass_idx, swapped=need_swap, settled=pass_idx)

        # 趟結束畫面：更新已就位數量為 pass_idx + 1
        draw(arr[:], i=-1, j=-1, pass_idx=pass_idx, swapped=any_swap, settled=pass_idx + 1)

        if not any_swap:
            # 已經有序，提前結束（但仍會在迴圈外另畫完成圖）
            break

    # 最終完成圖（全部綠色），顯示總比較次數
    draw(arr[:], i=-1, j=-1, pass_idx=max(0, n - 2), swapped=any_swap_overall, settled=n, title_suffix="｜完成")

if __name__ == "__main__":
    # 範例：可調整資料與輸出資料夾
    samples = {
        "一般": [5, 1, 4, 2, 8],
        "已排序": [1, 2, 3, 4, 5],
        "逆序": [9, 7, 5, 3, 1],
        "含重複": [3, 2, 3, 1, 2],
    }
    data = samples["一般"]
    save_bubble_sort_frames(data, outdir="frames_bubble_demo", ascending=True)
    print("已輸出到資料夾 frames_bubble_demo/（含初始、各步比較/趟結束、以及最終完成圖）")

已輸出到資料夾 frames_bubble_demo/（含初始、各步比較/趟結束、以及最終完成圖）


In [ ]:
from google.colab import files

!zip -r /content/frames_bubble_demo.zip /content/frames_bubble_demo

files.download('/content/frames_bubble_demo.zip')

  adding: content/frames_bubble_demo/ (stored 0%)
  adding: content/frames_bubble_demo/frame_0008.png (deflated 19%)
  adding: content/frames_bubble_demo/frame_0011.png (deflated 19%)
  adding: content/frames_bubble_demo/frame_0012.png (deflated 19%)
  adding: content/frames_bubble_demo/frame_0013.png (deflated 21%)
  adding: content/frames_bubble_demo/frame_0007.png (deflated 20%)
  adding: content/frames_bubble_demo/frame_0005.png (deflated 21%)
  adding: content/frames_bubble_demo/frame_0002.png (deflated 20%)
  adding: content/frames_bubble_demo/frame_0009.png (deflated 20%)
  adding: content/frames_bubble_demo/frame_0000.png (deflated 22%)
  adding: content/frames_bubble_demo/frame_0004.png (deflated 20%)
  adding: content/frames_bubble_demo/frame_0001.png (deflated 20%)
  adding: content/frames_bubble_demo/frame_0003.png (deflated 20%)
  adding: content/frames_bubble_demo/frame_0010.png (deflated 19%)
  adding: content/frames_bubble_demo/frame_0006.png (deflated 20%)


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
import os
from typing import List
import matplotlib.pyplot as plt

def save_insertion_sort_frames(a: List[int], outdir: str = "frames_insertion", ascending: bool = True):
    """
    逐步靜態可視化：插入排序（Insertion Sort）
    - 先輸出初始圖
    - 每個內層比較/移動都輸出一張圖
    - 每趟插入完成輸出一張圖
    - 最後再輸出一張「完成」總結圖（全部綠色）
    """
    os.makedirs(outdir, exist_ok=True)
    arr = a[:]
    n = len(arr)
    frame_id = 0
    comparisons = 0

    def draw(current, pass_idx, pos, j, key_index, action, sorted_len, title_suffix=""):
        """
        current: 當前陣列
        pass_idx: 第幾趟（處理第 pass_idx 索引；從 1 開始）
        pos: 正在處理的索引（與 pass_idx 語義一致）
        j: 內層比較中的左側索引（比較 arr[j] 與 key）
        key_index: 當前 key 的暫存位置（用橙色高亮）
        action: 描述字串（比較 / 右移 / 插入到位 / 初始 / 完成）
        sorted_len: 已排序區間長度（左側綠色數量）
        title_suffix: 額外標題說明（如｜初始、｜完成）
        """
        nonlocal frame_id
        plt.figure(figsize=(7.5, 4))
        bars = plt.bar(range(len(current)), current, color="#4C78A8")

        # 已排序區域（左側 sorted_len 個）標綠色
        for idx in range(sorted_len):
            if 0 <= idx < len(current):
                bars[idx].set_color("#72B07E")

        # 高亮 key 的暫存位置
        if 0 <= key_index < len(current):
            bars[key_index].set_color("#F28E2B")

        # 若正在比較，且 j 存在且不等於 key_index，也標橙色
        if 0 <= j < len(current) and j != key_index:
            bars[j].set_color("#F28E2B")

        plt.title("插入排序（Insertion Sort）逐步示意" + title_suffix)
        plt.xlabel("索引")
        plt.ylabel("數值")
        ymax = max(current) if current else 1
        plt.ylim(0, ymax * 1.2 if ymax > 0 else 1)

        if title_suffix == "｜初始":
            info = f"初始陣列 | 長度：{len(current)}"
        elif title_suffix == "｜完成":
            info = f"排序完成 | 總比較：{comparisons} | 已排序：{sorted_len} 個"
        else:
            info = f"處理第 {pass_idx} 趟（pos={pos}） | 動作：{action} | 累積比較：{comparisons} | 左側已排序：{sorted_len} 個"
        plt.text(0.02, 0.95, info, transform=plt.gca().transAxes, va='top', fontsize=10)

        plt.tight_layout()
        path = os.path.join(outdir, f"frame_{frame_id:04d}.png")
        plt.savefig(path, dpi=140)
        plt.close()
        frame_id += 1

    # 初始圖
    draw(arr[:], pass_idx=0, pos=0, j=-1, key_index=-1, action="初始", sorted_len=0, title_suffix="｜初始")

    if n <= 1:
        # 只有 0 或 1 個元素，初始即完成；輸出完成圖
        draw(arr[:], pass_idx=0, pos=0, j=-1, key_index=-1, action="完成", sorted_len=n, title_suffix="｜完成")
        return

    # 插入排序
    for k in range(1, n):
        key = arr[k]
        j = k - 1

        # 選取 key 的畫面（左側已有 k 個元素視為已排序）
        draw(arr[:], pass_idx=k, pos=k, j=j, key_index=k, action=f"選取 key={key}", sorted_len=k)

        # 內層比較與右移
        while j >= 0:
            comparisons += 1
            need_shift = (arr[j] > key) if ascending else (arr[j] < key)

            # 畫比較畫面（key 暫存於 j+1 位置，尚未覆寫）
            draw(arr[:], pass_idx=k, pos=k, j=j, key_index=j+1, action=f"比較 arr[{j}]={arr[j]} 與 key={key}", sorted_len=k)

            if need_shift:
                # 右移 arr[j] 至 arr[j+1]
                arr[j + 1] = arr[j]
                # 畫右移後的畫面（key 暫存位置視覺上在 j）
                draw(arr[:], pass_idx=k, pos=k, j=j, key_index=j, action=f"右移 arr[{j}] -> arr[{j+1}]", sorted_len=k)
                j -= 1
            else:
                break

        # 插入 key 到 j+1
        arr[j + 1] = key
        # 插入完成畫面：此時左側已排序長度擴大為 k+1，key 位於 j+1
        draw(arr[:], pass_idx=k, pos=k, j=j, key_index=j+1, action=f"插入 key={key} 至位置 {j+1}", sorted_len=k+1)

    # 演算法結束後，輸出最終完成圖（全部綠色）
    draw(arr[:], pass_idx=n-1, pos=n-1, j=-1, key_index=-1, action="完成", sorted_len=n, title_suffix="｜完成")

if __name__ == "__main__":
    samples = {
        "一般": [5, 1, 4, 2, 8],
        "已排序": [1, 2, 3, 4, 5],
        "逆序": [9, 7, 5, 3, 1],
        "含重複": [3, 2, 3, 1, 2],
    }
    data = samples["一般"]
    save_insertion_sort_frames(data, outdir="frames_insertion_demo", ascending=True)
    print("已輸出到資料夾 frames_insertion_demo/（含初始、比較/移動、插入完成，以及最終完成圖）")

已輸出到資料夾 frames_insertion_demo/（含初始、比較/移動、插入完成，以及最終完成圖）


In [ ]:
from google.colab import files

!zip -r /content/frames_insertion_demo.zip /content/frames_insertion_demo

files.download('/content/frames_insertion_demo.zip')

  adding: content/frames_insertion_demo/ (stored 0%)
  adding: content/frames_insertion_demo/frame_0008.png (deflated 19%)
  adding: content/frames_insertion_demo/frame_0011.png (deflated 19%)
  adding: content/frames_insertion_demo/frame_0012.png (deflated 19%)
  adding: content/frames_insertion_demo/frame_0013.png (deflated 19%)
  adding: content/frames_insertion_demo/frame_0007.png (deflated 19%)
  adding: content/frames_insertion_demo/frame_0020.png (deflated 21%)
  adding: content/frames_insertion_demo/frame_0016.png (deflated 19%)
  adding: content/frames_insertion_demo/frame_0015.png (deflated 19%)
  adding: content/frames_insertion_demo/frame_0005.png (deflated 19%)
  adding: content/frames_insertion_demo/frame_0002.png (deflated 19%)
  adding: content/frames_insertion_demo/frame_0009.png (deflated 19%)
  adding: content/frames_insertion_demo/frame_0019.png (deflated 18%)
  adding: content/frames_insertion_demo/frame_0000.png (deflated 22%)
  adding: content/frames_insertion_de

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>